In [1]:

# coding: utf-8

# In[1]:


import pandas as pd
import numpy as np
import heapq
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction import DictVectorizer
from bottleneck import argpartition

from math import ceil
from sklearn.neighbors import LSHForest


# In[2]:


class Rec():
    
    def __init__(self):
        self.anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
        self.anime_id = self.anime_tag_vector['anime_id'].as_matrix().tolist()
        self.anime_rating = self.anime_tag_vector['rating']
        del self.anime_tag_vector['anime_id']
        del self.anime_tag_vector['rating']
        self.anime_tag_vector = self.anime_tag_vector.as_matrix()

        # user instance: calculated rating to each tag
        self.instance = pd.read_csv('user_instances.csv')
        del self.instance['user_id']
        self.instance = self.instance[1:].as_matrix()
        
        
        # user actual rating to each watched anime
        self.rate = pd.read_csv('rating.csv')
        self.rate = self.rate.as_matrix().tolist()
        
        # cluster user to c groups
        self.c = 50
        self.kmeans = cluster.KMeans(n_clusters=self.c, max_iter=300, init='k-means++',n_init=10, verbose=True, n_jobs=-1).fit(self.instance)
        self.clus = self.kmeans.predict(self.instance)
        
        # user watched anime list    
        self.watch = np.array([[] for _ in self.instance]).tolist() ## TODO
        for r in self.rate:
            self.watch[r[0]-1].append(r[1])
        
        # tag
        self.animes = pd.read_csv("anime.csv")
        tags = []
        for genres in self.animes["genre"].fillna(""):
            for genre in str.split(genres, ", "):
                tags.append(genre)
        tags = sorted(list(set(tags))[1:]) # Remove ''
        v = DictVectorizer(sparse="False")
        genre = v.fit_transform(self.animes["genre"].fillna(", ".join(tags)).apply(lambda x: {i: 1 for i in str.split(x, ", ")}))
        self.genre = np.array(genre.todense())
        
        self.inverse_anime_id = {}
        for index, anime in enumerate(self.anime_id):
            self.inverse_anime_id[anime] = index
        
        # anime_candidates for each cluster (sorted)
        self.anime_candidates = []
        for i in range(self.c):
             
            print("Cluster ", i)
            
            users_in_clus = np.where(self.clus==i)[0]

            lshf = LSHForest(random_state=42)
            lshf.fit(self.instance[users_in_clus])  # X_train
            LSHForest(min_hash_match=4, n_candidates=50, n_estimators=10,
                          n_neighbors=5, radius=1.0, radius_cutoff_ratio=0.9,
                          random_state=42)
            num_representatives = ceil(len(users_in_clus)/100)
            distances, indices = lshf.kneighbors([self.kmeans.cluster_centers_[i]], n_neighbors=num_representatives) #X_test
            representative = list(set([watch_anime for represent in indices for watch_list in np.array(self.watch)[represent] \
                                       for watch_anime in watch_list if watch_anime in self.anime_id]))
            self.anime_candidates.append(sorted(representative, key=lambda x: self.anime_rating[self.anime_id.index(x)], reverse=True))

            
        self.anime_id_to_idx = {}
        for i in range(0, 12294):
            self.anime_id_to_idx.update({self.anime_id[i]:i})
        
        
    def avg_rating(self, train):
        
        rating_times = 0
        
        ratings = 0
        
        for r in train:
            anime_id = r[0]
            rating = r[1]
            if rating != -1:
                ratings += rating
                rating_times += 1
                

        np.seterr(divide='ignore', invalid='ignore')        
        if rating_times == 0:
            ratings_vector = 5
        else:
            ratings_vector = np.nan_to_num(ratings / rating_times)
        
        return ratings_vector
        
    
        
    def get_ranked_ids(self, datadata, k):
        
        # generate user instance from data
        data = dict(datadata)
        
        rating_times = 0
        rating_appeartime = np.zeros(43)
        ratings = np.zeros(43)
        avg_rate = self.avg_rating(datadata)
        
        for obj in datadata:
            
            anime_id = obj[0]
            r = obj[1]
        
            if r == -1:
                ratings += self.anime_tag_vector[self.anime_id_to_idx[anime_id]] * (10 - avg_rate)
            else:
                ratings += self.anime_tag_vector[self.anime_id_to_idx[anime_id]] * r
                
            rating_appeartime += self.anime_tag_vector[self.anime_id_to_idx[anime_id]]
        
        
        np.seterr(divide='ignore', invalid='ignore')        
        #ratings = np.nan_to_num(ratings / rating_appeartime)
        ins = np.nan_to_num(ratings / rating_appeartime)
        
        
        

        ins_watch = []
        for r in data.keys():
            ins_watch.append(r)
        ins_watch = set(ins_watch)
        print(ins_watch)
        

        # predict!!!!
        pred = self.kmeans.predict([ins])[0]
        #index = [i for i, x in enumerate(self.clus.tolist()) if x == pred]
        #anime_candidates = [w for i in index for w in self.watch[i] if w in self.anime_id]
        #anime_candidates = set(anime_candidates) - set(ins_watch)

        #k = min(k, len(self.anime_candidates))
        
        ans = []
        count = 0
        for cand in self.anime_candidates[pred]:
            if cand in ins_watch:
                continue
            if count == k:
                break
            ans.append(cand)
            count += 1
        
        return ans
        
        #return list(self.anime_candidates[pred]-set(ins_watch))[:k]

        # b = [self.anime_id.index(c) for c in self.anime_candidates[pred]]
        # r = self.anime_rating[b].as_matrix()
        # p = argpartition(r, len(self.anime_candidates) - k)
        # return [self.anime_id[pp] for pp in p[len(self.anime_candidates) - k:]]
        
        #return heapq.nlargest(k,self.anime_candidates[pred], key=lambda x: self.anime_rating[self.anime_id.index(x)])



In [2]:
rec = Rec()

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
start iteration
start iteration
start iteration
start iteration
start iteration
start iteration
start iteration
start iteration
done sorting
done sorting
done sorting
done sorting
done sorting
done sorting
done sorting
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 0, inertia 17517932.1341
Iteration 0, inertia 17435031.9945
start iteration
start iteration
Iteration 0, inertia 17352021.6675
done sorting
start iteration
done sorting
Iteration 0, inertia 17529382.744
done sorting
Iteration 0, inertia 17459783.7427
Iteration 0, inertia 17463043.4215
start iteration
start iteration
Iteration 0, inertia 17601518.6098
done sorting
start iteration
done sorting
start iteration
Iteration 0, inertia 17354782.9873


end inner loop
Iteration 12, inertia 15986035.3276
done sorting
start iteration
end inner loop
end inner loop
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 13, inertia 15956285.6652
start iteration
Iteration 12, inertia 15975229.2528
done sorting
start iteration
Iteration 13, inertia 15912622.4309
start iteration
Iteration 12, inertia 15947342.4094
done sorting
done sorting
start iteration
end inner loop
done sorting
Iteration 12, inertia 15973399.7962
start iteration
Iteration 11, inertia 15943423.5729
start iteration
done sorting
done sorting
Iteration 13, inertia 15923722.1976
start iteration
end inner loop
done sorting
end inner loop
end inner loop
start iteration
Iteration 13, inertia 15970725.444
end inner loop
end inner loop
done sorting
end inner loop
Iteration 14, inertia 15946785.7911
Iteration 12, inertia 15933365.5849
start iteration
done sorting
end inner loop
start iteration
done sorting
Iteration 13, inertia 15963531.2631
start iterat

end inner loop
start iteration
Iteration 25, inertia 15879085.6772
done sorting
done sorting
end inner loop
Iteration 26, inertia 15876864.9135
end inner loop
end inner loop
end inner loop
start iteration
done sorting
start iteration
done sorting
end inner loop
Iteration 24, inertia 15898547.191
start iteration
done sorting
end inner loop
Iteration 25, inertia 15861443.3089
start iteration
end inner loop
done sorting
end inner loop
end inner loop
Iteration 25, inertia 15866643.8102
start iteration
done sorting
Iteration 27, inertia 15859905.5162
start iteration
done sorting
Iteration 27, inertia 15874696.7184
start iteration
Iteration 25, inertia 15895885.4276
start iteration
Iteration 26, inertia 15876562.6982
done sorting
done sorting
start iteration
Iteration 24, inertia 15889685.7932
Iteration 26, inertia 15858096.0702
start iteration
start iteration
end inner loop
done sorting
done sorting
done sorting
Iteration 25, inertia 15874708.6696
end inner loop
end inner loop
end inner loo

end inner loop
Iteration 37, inertia 15864852.6485
start iteration
start iteration
end inner loop
done sorting
done sorting
Iteration 37, inertia 15857122.3952
end inner loop
end inner loop
Iteration 36, inertia 15871342.9659
Iteration 39, inertia 15857407.3617
start iteration
start iteration
done sorting
done sorting
start iteration
Iteration 38, inertia 15843416.1893
start iteration
Iteration 39, inertia 15865072.376
end inner loop
done sorting
Iteration 36, inertia 15850395.1029
done sorting
Iteration 38, inertia 15863265.3064
end inner loop
start iteration
done sorting
start iteration
start iteration
end inner loop
done sorting
done sorting
end inner loop
end inner loop
Iteration 38, inertia 15856428.382
start iteration
end inner loop
done sorting
end inner loop
Iteration 41, inertia 15844276.0811
end inner loop
Iteration 40, inertia 15856887.465
start iteration
start iteration
Iteration 39, inertia 15842906.698
start iteration
Iteration 37, inertia 15870848.4418
done sorting
done 

end inner loop
Iteration 50, inertia 15867176.4807
start iteration
done sorting
Iteration 51, inertia 15851265.0402
start iteration
Iteration 52, inertia 15859163.6577
start iteration
Iteration 51, inertia 15837499.6836
start iteration
done sorting
done sorting
done sorting
end inner loop
end inner loop
end inner loop
Iteration 50, inertia 15847247.5936
Iteration 53, inertia 15833008.326
start iteration
end inner loop
done sorting
Iteration 48, inertia 15839403.3633
start iteration
done sorting
Iteration 50, inertia 15851327.469
end inner loop
start iteration
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 52, inertia 15850656.291
Iteration 51, inertia 15866792.0453
start iteration
Iteration 52, inertia 15837137.5286
start iteration
Iteration 54, inertia 15832313.5519
end inner loop
done sorting
start iteration
done sorting
start iteration
done sorting
done sorting
end inner loop
end inner loop
end inner loop
Iteration 49, inertia 15838660.6765
start i

...

done sorting
Iteration 120, inertia 15838449.9213
end inner loop
end inner loop
start iteration
done sorting
Iteration 119, inertia 15819403.5771
start iteration
end inner loop
done sorting
Iteration 10, inertia 15969908.0281
Iteration 115, inertia 15833875.6281
end inner loop
Iteration 117, inertia 15837901.9493
start iteration
start iteration
done sorting
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 118, inertia 15849798.4973
Iteration 121, inertia 15838406.8389
start iteration
done sorting
start iteration
Iteration 116, inertia 15841789.2084
end inner loop
done sorting
Iteration 120, inertia 15819397.8722
start iteration
done sorting
Iteration 118, inertia 15837900.7373
start iteration
start iteration
end inner loop
done sorting
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 116, inertia 15833836.4006
start iteration
done sorting
Iteration 121, inertia 15819390.1634
end inner loop
start iteration
Iteration 11, 

done sorting
Iteration 131, inertia 15849766.95
end inner loop
end inner loop
center shift 2.251578e-02 within tolerance 1.075880e-03
end inner loop
Iteration 134, inertia 15819212.8438
Iteration 135, inertia 15837998.1113
start iteration
done sorting
start iteration
end inner loop
Iteration 132, inertia 15841622.3282
start iteration
done sorting
done sorting
Iteration 25, inertia 15862788.2432
end inner loop
Iteration 131, inertia 15833487.5859
end inner loop
start iteration
start iteration
done sorting
done sorting
end inner loop
Iteration 135, inertia 15819206.1192
end inner loop
Iteration 8, inertia 15968884.7504
start iteration
start iteration
Iteration 133, inertia 15841621.1474
start iteration
done sorting
done sorting
done sorting
Iteration 26, inertia 15859074.5659
Iteration 136, inertia 15837953.3794
end inner loop
end inner loop
start iteration
end inner loop
start iteration
done sorting
Iteration 132, inertia 15833482.9928
done sorting
start iteration
end inner loop
done so

done sorting
Iteration 49, inertia 15823711.4645
end inner loop
start iteration
done sorting
end inner loop
Iteration 155, inertia 15818619.5364
start iteration
done sorting
Iteration 29, inertia 15841137.5604
start iteration
done sorting
end inner loop
end inner loop
Iteration 50, inertia 15823151.9906
start iteration
Iteration 30, inertia 15840789.0257
done sorting
start iteration
done sorting
Iteration 156, inertia 15818573.1102
end inner loop
start iteration
end inner loop
done sorting
end inner loop
Iteration 51, inertia 15822603.8227
start iteration
done sorting
end inner loop
Iteration 31, inertia 15840383.5699
start iteration
Iteration 157, inertia 15818543.5622
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 52, inertia 15822182.3847
start iteration
done sorting
end inner loop
Iteration 158, inertia 15818516.6744
Iteration 53, inertia 15821828.0679
Iteration 32, inertia 15840002.8461
start iteration
start iteration
done sorting
start iteration

Iteration 85, inertia 15818428.0871
start iteration
Iteration 62, inertia 15831167.9989
start iteration
done sorting
done sorting
end inner loop
end inner loop
end inner loop
Iteration 86, inertia 15818371.3046
start iteration
Iteration 63, inertia 15830410.2288
done sorting
start iteration
Iteration 188, inertia 15818374.7038
done sorting
start iteration
end inner loop
done sorting
end inner loop
end inner loop
Iteration 87, inertia 15818319.8557
start iteration
done sorting
Iteration 64, inertia 15829811.2302
end inner loop
start iteration
done sorting
Iteration 189, inertia 15818372.9769
start iteration
done sorting
end inner loop
Iteration 88, inertia 15818272.8688
end inner loop
start iteration
done sorting
end inner loop
Iteration 65, inertia 15829215.2981
start iteration
done sorting
Iteration 190, inertia 15818371.8917
start iteration
done sorting
Iteration 89, inertia 15818235.1697
end inner loop
done sorting
start iteration
end inner loop
end inner loop
Iteration 90, inertia 

start iteration
done sorting
end inner loop
Iteration 123, inertia 15820145.732
start iteration
done sorting
end inner loop
Iteration 124, inertia 15820138.6314
start iteration
done sorting
end inner loop
Iteration 125, inertia 15820127.4203
start iteration
done sorting
end inner loop
Iteration 126, inertia 15820119.9143
start iteration
done sorting
end inner loop
Iteration 127, inertia 15820109.011
start iteration
done sorting
end inner loop
Iteration 128, inertia 15820104.3864
start iteration
done sorting
end inner loop
Iteration 129, inertia 15820102.5761
start iteration
done sorting
end inner loop
Iteration 130, inertia 15820101.6612
start iteration
done sorting
end inner loop
Iteration 131, inertia 15820100.4702
start iteration
done sorting
end inner loop
Iteration 132, inertia 15820099.8232
start iteration
done sorting
end inner loop
Iteration 133, inertia 15820099.8232
center shift 0.000000e+00 within tolerance 1.075880e-03
Cluster  0
Cluster  1
Cluster  2
Cluster  3
Cluster  4


In [3]:
import math
for i in range(rec.c):
    for cand in rec.anime_candidates[i]:
        if math.isnan(rec.anime_rating[rec.anime_id.index(cand)]):
            print((cand, rec.anime_rating[rec.anime_id.index(cand)]))

In [4]:

import pandas as pd
import numpy as np
import heapq
from sklearn import cluster
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction import DictVectorizer
from bottleneck import argpartition

from math import ceil
from sklearn.neighbors import LSHForest


# In[2]:


class Rec():
    
    def __init__(self):
        self.anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
        self.anime_id = self.anime_tag_vector['anime_id'].as_matrix().tolist()
        self.anime_rating = self.anime_tag_vector['rating']
        del self.anime_tag_vector['anime_id']
        del self.anime_tag_vector['rating']
        self.anime_tag_vector = self.anime_tag_vector.as_matrix()

        # user instance: calculated rating to each tag
        self.instance = pd.read_csv('user_instances.csv')
        del self.instance['user_id']
        self.instance = self.instance[1:].as_matrix()
        
        
        # user actual rating to each watched anime
        self.rate = pd.read_csv('rating.csv')
        self.rate = self.rate.as_matrix().tolist()
        
        # cluster user to c groups
        self.c = 400
        self.percentage = 100
        self.kmeans = cluster.KMeans(n_clusters=self.c, max_iter=300, init='k-means++',n_init=10, verbose=True, n_jobs=-1).fit(self.instance)
        self.clus = self.kmeans.predict(self.instance)
        
        # user watched anime list    
        self.watch = np.array([[] for _ in self.instance]).tolist() ## TODO
        for r in self.rate:
            self.watch[r[0]-1].append(r[1])
        
        # tag
        self.animes = pd.read_csv("anime.csv")
        tags = []
        for genres in self.animes["genre"].fillna(""):
            for genre in str.split(genres, ", "):
                tags.append(genre)
        tags = sorted(list(set(tags))[1:]) # Remove ''
        v = DictVectorizer(sparse="False")
        genre = v.fit_transform(self.animes["genre"].fillna(", ".join(tags)).apply(lambda x: {i: 1 for i in str.split(x, ", ")}))
        self.genre = np.array(genre.todense())
        
        self.inverse_anime_id = {}
        for index, anime in enumerate(self.anime_id):
            self.inverse_anime_id[anime] = index
        
        # anime_candidates for each cluster (sorted)
        self.anime_candidates = []
        for i in range(self.c):
             
            print("Cluster ", i)
            
            users_in_clus = np.where(self.clus==i)[0]

            lshf = LSHForest(random_state=42)
            lshf.fit(self.instance[users_in_clus])  # X_train
            LSHForest(min_hash_match=4, n_candidates=50, n_estimators=10,
                          n_neighbors=5, radius=1.0, radius_cutoff_ratio=0.9,
                          random_state=42)
            num_representatives = ceil(len(users_in_clus)/ self.percentage)
            distances, indices = lshf.kneighbors([self.kmeans.cluster_centers_[i]], n_neighbors=num_representatives) #X_test
            representative = list(set([watch_anime for represent in indices for watch_list in np.array(self.watch)[represent] \
                                       for watch_anime in watch_list if watch_anime in self.anime_id]))
            self.anime_candidates.append(sorted(representative, key=lambda x: self.anime_rating[self.anime_id.index(x)], reverse=True))

            
        self.anime_id_to_idx = {}
        for i in range(0, 12294):
            self.anime_id_to_idx.update({self.anime_id[i]:i})
        
        
    def avg_rating(self, train):
        
        rating_times = 0
        
        ratings = 0
        
        for r in train:
            anime_id = r[0]
            rating = r[1]
            if rating != -1:
                ratings += rating
                rating_times += 1
                

        np.seterr(divide='ignore', invalid='ignore')        
        if rating_times == 0:
            ratings_vector = 5
        else:
            ratings_vector = np.nan_to_num(ratings / rating_times)
        
        return ratings_vector
        
    
        
    def get_ranked_ids(self, datadata, k):
        
        # generate user instance from data
        data = dict(datadata)
        
        rating_times = 0
        rating_appeartime = np.zeros(43)
        ratings = np.zeros(43)
        avg_rate = self.avg_rating(datadata)
        
        for obj in datadata:
            
            anime_id = obj[0]
            r = obj[1]
        
            if r == -1:
                ratings += self.anime_tag_vector[self.anime_id_to_idx[anime_id]] * (10 - avg_rate)
            else:
                ratings += self.anime_tag_vector[self.anime_id_to_idx[anime_id]] * r
                
            rating_appeartime += self.anime_tag_vector[self.anime_id_to_idx[anime_id]]
        
        
        np.seterr(divide='ignore', invalid='ignore')        
        #ratings = np.nan_to_num(ratings / rating_appeartime)
        ins = np.nan_to_num(ratings / rating_appeartime)
        
        
        

        ins_watch = []
        for r in data.keys():
            ins_watch.append(r)
        ins_watch = set(ins_watch)
        #print(ins_watch)
        

        # predict!!!!
        pred = self.kmeans.predict([ins])[0]
        #index = [i for i, x in enumerate(self.clus.tolist()) if x == pred]
        #anime_candidates = [w for i in index for w in self.watch[i] if w in self.anime_id]
        #anime_candidates = set(anime_candidates) - set(ins_watch)

        #k = min(k, len(self.anime_candidates))
        
        ans = []
        count = 0
        for cand in self.anime_candidates[pred]:
            if cand in ins_watch:
                continue
            if count == k:
                break
            ans.append(cand)
            count += 1
        
        return ans
        

In [5]:
rec = Rec()

Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
Initialization complete
start iteration
done sorting
start iteration
start iteration
done sorting
done sorting
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
start iteration
done sorting
start iteration
Iteration 0, inertia 13064873.4991
start iteration
done sorting
done sorting
end inner loop
Iteration 0, inertia 13077337.104
start iteration
start iteration
done sorting
done sorting
Iteration 0, inertia 13025874.6337
start iteration
done sorting
Iteration 0, inertia 13073270.5699
start iteration
done sorting
end inner loop
end inner loop
Iteration 0, inertia 13032284.7938
start iteration
done sorting
end inner loop
end inner loop
start iteration
done sorting
end inner loop
Iteration 0, inertia 13091088.3688
start iteration
done sorting
Iteration 0, inertia 13102798.9362
start it

end inner loop
end inner loop
end inner loop
Iteration 11, inertia 12087335.746
start iteration
done sorting
end inner loop
Iteration 13, inertia 12067166.5371
start iteration
done sorting
end inner loop
Iteration 11, inertia 12088844.3519
start iteration
done sorting
end inner loop
Iteration 13, inertia 12066280.1786
end inner loop
start iteration
done sorting
Iteration 12, inertia 12083967.8956
start iteration
done sorting
Iteration 13, inertia 12065764.7207
end inner loop
start iteration
done sorting
end inner loop
Iteration 12, inertia 12085723.0803
start iteration
Iteration 13, inertia 12077559.2633
done sorting
start iteration
done sorting
end inner loop
Iteration 12, inertia 12078660.68
start iteration
done sorting
end inner loop
end inner loop
Iteration 14, inertia 12059632.8161
start iteration
done sorting
end inner loop
end inner loop
Iteration 12, inertia 12078723.4098
Iteration 14, inertia 12060690.1037
start iteration
done sorting
start iteration
done sorting
Iteration 13,

Iteration 24, inertia 12024274.7052
start iteration
done sorting
Iteration 24, inertia 12029148.8057
start iteration
end inner loop
done sorting
Iteration 24, inertia 12031450.7654
Iteration 26, inertia 12017048.7824
start iteration
start iteration
done sorting
done sorting
end inner loop
end inner loop
end inner loop
Iteration 26, inertia 12021788.4522
start iteration
done sorting
end inner loop
end inner loop
Iteration 24, inertia 12025313.1251
end inner loop
start iteration
done sorting
Iteration 26, inertia 12013355.4672
start iteration
done sorting
Iteration 25, inertia 12021414.0458
start iteration
done sorting
Iteration 26, inertia 12035036.8336
start iteration
end inner loop
done sorting
Iteration 25, inertia 12027142.1173
start iteration
Iteration 27, inertia 12015059.982
done sorting
start iteration
done sorting
end inner loop
end inner loop
Iteration 25, inertia 12029111.9107
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 27, inertia 1202

start iteration
done sorting
Iteration 39, inertia 12002715.549
start iteration
done sorting
Iteration 38, inertia 11999127.5543
end inner loop
start iteration
done sorting
Iteration 39, inertia 12007978.839
start iteration
done sorting
end inner loop
Iteration 38, inertia 12004214.6492
Iteration 37, inertia 12016553.7444
start iteration
start iteration
done sorting
done sorting
Iteration 38, inertia 12022805.4613
start iteration
end inner loop
done sorting
end inner loop
end inner loop
Iteration 37, inertia 12011795.1182
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 37, inertia 12005218.641
start iteration
done sorting
Iteration 40, inertia 12002333.9491
start iteration
Iteration 39, inertia 11998604.3493
start iteration
done sorting
done sorting
end inner loop
Iteration 40, inertia 12007420.0574
start iteration
done sorting
end inner loop
end inner loop
Iteration 38, inertia 12016030.2552
start iteration
end inner loop
done sorting
Iteration 39, 

end inner loop
end inner loop
end inner loop
Iteration 49, inertia 12003969.5247
start iteration
done sorting
end inner loop
end inner loop
Iteration 52, inertia 12004313.3538
start iteration
done sorting
Iteration 51, inertia 11994561.9406
start iteration
done sorting
Iteration 49, inertia 11998136.1669
start iteration
done sorting
Iteration 50, inertia 12010834.3521
end inner loop
start iteration
done sorting
Iteration 51, inertia 12014572.7886
start iteration
done sorting
Iteration 53, inertia 11999408.6102
start iteration
end inner loop
done sorting
Iteration 51, inertia 11995298.495
end inner loop
start iteration
done sorting
end inner loop
end inner loop
Iteration 53, inertia 12004252.8243
start iteration
Iteration 50, inertia 12003535.6618
end inner loop
done sorting
start iteration
end inner loop
done sorting
end inner loop
Iteration 52, inertia 11994216.2088
start iteration
done sorting
end inner loop
end inner loop
Iteration 50, inertia 11997611.5347
Iteration 52, inertia 120

...

Iteration 76, inertia 11988225.006
start iteration
Initialization complete
done sorting
end inner loop
Iteration 77, inertia 12008149.6312
start iteration
done sorting
end inner loop
end inner loop
Iteration 78, inertia 11986325.4332
start iteration
end inner loop
Iteration 77, inertia 11994846.0993
done sorting
start iteration
done sorting
end inner loop
Iteration 79, inertia 12009220.6162
start iteration
Iteration 78, inertia 11991362.7136
done sorting
start iteration
done sorting
end inner loop
Iteration 81, inertia 11996257.1711
end inner loop
Iteration 77, inertia 11988104.539
start iteration
done sorting
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 78, inertia 12008124.872
start iteration
done sorting
Iteration 79, inertia 11986195.3932
start iteration
done sorting
Iteration 78, inertia 11994767.2757
start iteration
done sorting
Iteration 80, inertia 12009102.4484
end inner loop
start iteration
end inner loop
done sorting
Iter

done sorting
Iteration 92, inertia 12007594.9633
start iteration
done sorting
Iteration 7, inertia 12157685.6031
end inner loop
start iteration
done sorting
Iteration 91, inertia 12008034.4024
start iteration
done sorting
end inner loop
Iteration 94, inertia 11995822.818
start iteration
done sorting
Iteration 90, inertia 11986779.8969
start iteration
end inner loop
done sorting
end inner loop
end inner loop
Iteration 91, inertia 11989912.747
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 91, inertia 11994313.7161
Iteration 92, inertia 11984383.9674
start iteration
start iteration
done sorting
done sorting
Iteration 93, inertia 12007421.1863
start iteration
Iteration 92, inertia 12008027.065
done sorting
start iteration
done sorting
end inner loop
Iteration 8, inertia 12138341.8534
start iteration
done sorting
Iteration 95, inertia 11995794.3555
start iteration
end inner loop
done sorting
Iteration 91, inertia 11986750.6677
end inner l

start iteration
done sorting
Iteration 106, inertia 11993630.5589
start iteration
done sorting
Iteration 107, inertia 11983190.3824
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 21, inertia 12028691.4886
Iteration 107, inertia 11989415.7599
start iteration
done sorting
start iteration
done sorting
Iteration 107, inertia 11993606.8941
start iteration
done sorting
Iteration 108, inertia 11983168.8115
start iteration
done sorting
end inner loop
end inner loop
end inner loop
end inner loop
Iteration 108, inertia 11989398.4365
start iteration
done sorting
Iteration 22, inertia 12025291.4082
start iteration
done sorting
Iteration 108, inertia 11993593.3253
start iteration
done sorting
end inner loop
Iteration 109, inertia 11983147.8973
start iteration
done sorting
end inner loop
end inner loop
end inner loop
Iteration 109, inertia 11989393.704
start iteration
done sorting
Iteration 109, inertia 11993571.7435
start iteration
end inner loop


start iteration
done sorting
Iteration 54, inertia 11990903.701
start iteration
done sorting
end inner loop
end inner loop
Iteration 55, inertia 11990584.6576
start iteration
done sorting
Iteration 22, inertia 12035810.049
start iteration
done sorting
end inner loop
end inner loop
Iteration 56, inertia 11990254.6362
start iteration
done sorting
Iteration 23, inertia 12033661.3218
start iteration
done sorting
end inner loop
end inner loop
Iteration 57, inertia 11989935.7745
start iteration
done sorting
end inner loop
Iteration 24, inertia 12031728.8837
start iteration
done sorting
end inner loop
Iteration 58, inertia 11989677.6303
start iteration
done sorting
end inner loop
Iteration 25, inertia 12029758.4503
start iteration
done sorting
Iteration 59, inertia 11989400.7119
start iteration
end inner loop
done sorting
end inner loop
Iteration 26, inertia 12027919.8929
start iteration
done sorting
Iteration 60, inertia 11989194.1438
start iteration
done sorting
end inner loop
end inner loo

Cluster  60
Cluster  61
Cluster  62
Cluster  63
Cluster  64
Cluster  65
Cluster  66
Cluster  67
Cluster  68
Cluster  69
Cluster  70
Cluster  71
Cluster  72
Cluster  73
Cluster  74
Cluster  75
Cluster  76
Cluster  77
Cluster  78
Cluster  79
Cluster  80
Cluster  81
Cluster  82
Cluster  83
Cluster  84
Cluster  85
Cluster  86
Cluster  87
Cluster  88
Cluster  89
Cluster  90
Cluster  91
Cluster  92
Cluster  93
Cluster  94
Cluster  95
Cluster  96
Cluster  97
Cluster  98
Cluster  99
Cluster  100
Cluster  101
Cluster  102
Cluster  103
Cluster  104
Cluster  105
Cluster  106
Cluster  107
Cluster  108
Cluster  109
Cluster  110
Cluster  111
Cluster  112
Cluster  113
Cluster  114
Cluster  115
Cluster  116
Cluster  117
Cluster  118
Cluster  119
Cluster  120
Cluster  121
Cluster  122
Cluster  123
Cluster  124
Cluster  125
Cluster  126
Cluster  127
Cluster  128
Cluster  129
Cluster  130
Cluster  131
Cluster  132
Cluster  133
Cluster  134
Cluster  135
Cluster  136
Cluster  137
Cluster  138
Cluster  139


In [12]:
ag = pd.read_csv('ller.csv')
ag_data = [(a[0],a[1]) for a in ag.as_matrix()]
recommend = rec.get_ranked_ids(ag_data, 50)

anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
anime_id = anime_tag_vector['anime_id'].as_matrix().tolist()

animes = pd.read_csv("anime.csv")
[animes['name'][anime_id.index(r)] for r in recommend]

['Hunter x Hunter (2011)',
 'Tengen Toppa Gurren Lagann',
 'Fate/Zero 2nd Season',
 'Death Note',
 'Kiseijuu: Sei no Kakuritsu',
 'Hellsing Ultimate',
 'Gintama Movie: Shinyaku Benizakura-hen',
 'Shingeki no Kyojin',
 'Baccano!',
 'Fate/Zero',
 'Samurai Champloo',
 'Psycho-Pass',
 'Katanagatari',
 'No Game No Life',
 'Ghost in the Shell: Stand Alone Complex',
 'Kuroko no Basket',
 'Toradora!',
 'Stranger: Mukou Hadan',
 'One Piece Film: Strong World',
 'Berserk',
 'Mobile Suit Gundam Unicorn',
 'Ouran Koukou Host Club',
 'Angel Beats!',
 'Durarara!!',
 'Fullmetal Alchemist',
 'Trigun',
 'Dragon Ball Z',
 'Sekaiichi Hatsukoi 2',
 'Summer Wars',
 'Baccano! Specials',
 'Shijou Saikyou no Deshi Kenichi',
 'Darker than Black: Kuro no Keiyakusha',
 'Mobile Suit Gundam 00',
 'Gekkan Shoujo Nozaki-kun',
 'Fairy Tail',
 'Mobile Suit Gundam 00 Second Season',
 'Ansatsu Kyoushitsu (TV)',
 'Eureka Seven',
 'Noragami',
 'Black Lagoon',
 'Phantom: Requiem for the Phantom',
 'Akira',
 'Sekaiichi Hats

In [7]:
animes = pd.read_csv("anime.csv")
[animes['name'][anime_id.index(r)] for r in recommend]

NameError: name 'anime_id' is not defined

In [ ]:
anime_tag_vector = pd.read_csv('anime_tag_vector.csv')
anime_id = anime_tag_vector['anime_id'].as_matrix().tolist()

In [ ]:
[anime_id.index(r) for r in recommend]